In [ ]:
#in case we need this
%load_ext autoreload
%autoreload 2

import torch
import torch.nn as nn
import torch.optim as optim
from model_defs import FTTransformer  # WE are importing class here
from torch.utils.data import Dataset, DataLoader
from data_utils import ACSDataset
from torch.utils.data import DataLoader

from sklearn.preprocessing import LabelEncoder, StandardScaler


In [ ]:

# set device for gpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device.type == "cuda":
    print(f"found 1 gpu, using {device}")
else:
    print(f"found no gpu, using {device}")

In [ ]:
train_df = pd.read_csv("preprocessing_data/train_data_final_feat_no_preprocessing.csv")

train_df = train_df.drop(columns=['POVIP'])

In [ ]:
# continuous
cont_cols = ['AGEP', 'WKHP', 'year']

#  categorical
cat_cols = [
    'CIT', 'DIS', 'ENG', 'ESR', 'HICOV', 'LANP', 'LANX', 'MAR',
    'MIG', 'MSP', 'NATIVITY', 'OCCP', 'POBP', 'PRIVCOV', 'PUBCOV',
    'PUMA', 'RAC1P', 'SCHL', 'SEX', 'WKL', 'WRK'
]
#target
target_col = 'poverty_risk_score'

In [ ]:
for col in cat_cols:
    train_df[col] = train_df[col].fillna(-1)
for col in cont_cols:
    train_df[col] = train_df[col].fillna(train_df[col].median())

In [ ]:
#  Label Encode Categoricals
# we add +1 later in the model to handle that -1 Missing code
for col in cat_cols:
    le = LabelEncoder()
    train_df[col] = le.fit_transform(train_df[col].astype(str))

In [ ]:
#scaling the cont columns here using standard scalar
scaler = StandardScaler()
train_df[cont_cols] = scaler.fit_transform(train_df[cont_cols])

In [ ]:
train_ds = ACSDataset(train_df, cat_cols, cont_cols, 'poverty_risk')
#  DataLoader handles shuffling/ batching
train_loader = DataLoader(train_ds, batch_size=128, shuffle=True)


In [ ]:

cards = [df[col].nunique() for col in cat_cols]

model = FTTransformer(
    cat_cardinalities=cards,
    n_cont=len(cont_cols),
    embed_dim=32
).to(device)

criterion = torch.nn.CrossEntropyLoss()
#is this the best optimizer? keep reading
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
model.train()
for epoch in range(10):
    #o
    epoch_loss = 0
    for x_cat, x_cont, y in train_loader:
        x_cat, x_cont, y = x_cat.to(device), x_cont.to(device), y.to(device)

        optimizer.zero_grad()
        output = model(x_cat, x_cont)
        loss = criterion(output, y)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    print(f"Epoch {epoch+1} | Avg Loss: {epoch_loss/len(train_loader):.4f}")